In [21]:
from scripts import utils

import os
from pprint import pprint
import openai

openai.api_key = utils.get_openai_api_key()

In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["pdfs/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [6]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]), "\n")
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'> 

Doc ID: 0c5ae720-8d88-4cbb-a18b-d21708dc641b
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


In [12]:
documents[1].text

'PAGE 2"AI is the new \nelectricity. It will \ntransform and improve \nall areas of human life."\nAndrew Ng'

### BASIC RAG PIPELINE

In [13]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [17]:
# print(document.text)

In [18]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)

index = VectorStoreIndex.from_documents([document], service_context=service_context)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [19]:
query_engine = index.as_query_engine()

In [22]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
pprint(str(response))

('When finding projects to build your experience, there are several steps you '
 'can take. First, you can join existing projects by asking to join someone '
 "else's project if they have an idea. Additionally, you can continue reading, "
 'taking courses, and talking to domain experts to come up with new ideas. It '
 'is also helpful to focus on a specific application area where machine '
 'learning has not yet been applied. This can give you the opportunity to '
 'explore unique and creative applications that no one else has done yet. '
 'Finally, you can develop a side hustle or personal project that may not '
 'initially be part of your job but can stir your creative juices and '
 'strengthen bonds with collaborators.')


### Evaluation setup using TruLens

In [23]:
eval_questions = []
with open("texts/eval_questions.txt", "r") as file:
    for line in file:
        # Remove newline character 
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [24]:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [25]:
pprint(eval_questions)

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter '
 'syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'What is the right AI job for me?']


In [32]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

openai request failed <class 'openai.RateLimitError'>=Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-cbE5rpB12OCY5Q0iIRAY6hNo on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}. Retries remaining=3.


In [27]:
from scripts.utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine, app_id="Direct Query Engine")

In [34]:
# import time

# with tru_recorder as recording:
#     for question in eval_questions[:2]:
#         response = query_engine.query(question)
#         time.sleep(20)